# Medellín Chicharron
## Peer-graded Assignment: Battle of Neighbourhoods - Week 1

###### By: Marco Cruz Dawkins


## Part 1 - A description of the problem and a discussion of the background.
### 1.1 Description of the Problem
Andrea thinks there is no specialized chicharron restaurant in Medellin knowing that this city culturally loves this dish because it is the best part of the Bandeja Paisa. But she does not know where would be the best spot for this restaurant having in mind that is a new category of the restaurant in the city.

### 1.2 Discussion of the Background
Andrea family have a unique recipe for this dish that they are willing to share and she also is taking a trip to get trained in Barranquilla, Colombia that has more than 10 specialized chicharron restaurant that works 24/7 so her venture has the best chicharron once it's launched.

### 1.3 Target Audience
 Considering that Medellinenses love to party and after the party eats something greasy before they head home or to continue the party, is time for the city to get their first chicharron restaurant having in mind that they love it, male and female between 10 and 59 years are the perfect range of age to target.

## Part 2 - A description of the data and how it will be used to solve the problem 
 This project will rely on public data from Wikipedia and Foursquare.
 
 ### How data will be used to solve the problem
 The data from the datasets from Wikipedia and Foursquare will be explored by considering the venues within the neighbourhood of Medellin Comunas. These areas' restaurants, bars, offices and hotels would be checked in terms of the amount within a certain mile radius. Due to Foursquare restrictions, the number of venues will be limited to 500 venues.  

In [3]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import os
from sklearn.cluster import KMeans
import folium
from geopy.geocoders import Nominatim 
import matplotlib.cm as cm
import matplotlib.colors as colors

In [4]:
List_url = "https://es.wikipedia.org/wiki/Anexo:Barrios_de_Medell%C3%ADn"
source = requests.get(List_url).text

In [5]:
soup = BeautifulSoup(source, 'xml')
#print(soup.prettify())

In [6]:
links= soup.find_all('a')
links

[<a id="top"/>,
 <a class="mw-jump-link" href="#mw-head">Ir a la navegación</a>,
 <a class="mw-jump-link" href="#p-search">Ir a la búsqueda</a>,
 <a href="/wiki/Medell%C3%ADn" title="Medellín">Medellín</a>,
 <a href="/wiki/Colombia" title="Colombia">Colombia</a>,
 <a href="/wiki/Barrio" title="Barrio">barrios</a>,
 <a href="/wiki/Comunas_de_Medell%C3%ADn" title="Comunas de Medellín">comunas</a>,
 <a class="external text" href="https://www.medellin.gov.co/" rel="nofollow">Alcaldía de Medellín</a>,
 <a class="external text" href="http://www.concejodemedellin.gov.co/" rel="nofollow">Consejo Municipal de Medellín</a>,
 <a href="#cite_note-1"><span class="corchete-llamada">[</span>1<span class="corchete-llamada">]</span></a>,
 <a href="#Comuna_1_-_Popular"><span class="tocnumber">1</span> <span class="toctext">Comuna 1 - Popular</span></a>,
 <a href="#Comuna_2_-_Santa_Cruz"><span class="tocnumber">2</span> <span class="toctext">Comuna 2 - Santa Cruz</span></a>,
 <a href="#Comuna_3_-_Manriqu

In [7]:
comunas= []
comuna= soup.get_text()[3677:4074].split("\n")
for string in comuna:
    comunas.append(string[2:])
comuna =[]
for string in comunas:
    string = string.lstrip()
    comuna.append(string)
print(comuna)

['Comuna 1 - Popular', 'Comuna 2 - Santa Cruz', 'Comuna 3 - Manrique', 'Comuna 4 - Aranjuez', 'Comuna 5 - Castilla', 'Comuna 6 - Doce de Octubre', 'Comuna 7 - Robledo', 'Comuna 8 - Villa Hermosa', 'Comuna 9 - Buenos Aires', 'Comuna 10 - La Candelaria', 'Comuna 11 - Laureles-Estadio', 'Comuna 12 - La América', 'Comuna 13 - San Javier', 'Comuna 14 - El Poblado', 'Comuna 15 - Guayabal', 'Comuna 16 - Belén']


In [8]:
df = pd.DataFrame()
df['Comunas'] = comuna
df

,Comunas
0,Comuna 1 - Popular
1,Comuna 2 - Santa Cruz
2,Comuna 3 - Manrique
3,Comuna 4 - Aranjuez
4,Comuna 5 - Castilla
5,Comuna 6 - Doce de Octubre
6,Comuna 7 - Robledo
7,Comuna 8 - Villa Hermosa
8,Comuna 9 - Buenos Aires
9,Comuna 10 - La Candelaria


In [9]:
def get_coords_local(comunas, output_as='center'):
    """
    get the bounding box of a comunuty in WGS84 given its name
    Parameters
    ----------
    localidad : str
        name of the country in english and lowercase
    output_as : 'str
        chose from 'boundingbox' or 'center'. 
         - 'boundingbox' for [latmin, latmax, lonmin, lonmax]
         - 'center' for [latcenter, loncenter]
    Returns
    -------
    output : list
        list with coordinates as str
    """
    # create url
    url = '{0}{1}{2}'.format('http://nominatim.openstreetmap.org/search.php?q=',
                             comunas +', Medellín, Antioquia',
                             '&format=json&polygon=0')
    response = requests.get(url).json()[0]

    # parse response to list
    if output_as == 'boundingbox':
        lst = response[output_as]
        output = [float(i) for i in lst]
    if output_as == 'center':
        lst = [response.get(key) for key in ['lat','lon']]
        output = [float(i) for i in lst]
    return output

In [10]:
df2 = df.copy()

latitudeCln = []
longitudeCln = []
for index, row in df2.iterrows():
    print(row[0])
    lat, long = get_coords_local(comunas=row[0], output_as='center')
    latitudeCln.append(lat)
    longitudeCln.append(long)

df2['Latitude'] = latitudeCln
df2['Longitude'] = longitudeCln

df2.shape

Comuna 1 - Popular
Comuna 2 - Santa Cruz
Comuna 3 - Manrique
Comuna 4 - Aranjuez
Comuna 5 - Castilla
Comuna 6 - Doce de Octubre
Comuna 7 - Robledo
Comuna 8 - Villa Hermosa
Comuna 9 - Buenos Aires
Comuna 10 - La Candelaria
Comuna 11 - Laureles-Estadio
Comuna 12 - La América
Comuna 13 - San Javier
Comuna 14 - El Poblado
Comuna 15 - Guayabal
Comuna 16 - Belén


(16, 3)

In [11]:
df2

,Comunas,Latitude,Longitude
0,Comuna 1 - Popular,6.293344,-75.544825
1,Comuna 2 - Santa Cruz,6.296956,-75.554852
2,Comuna 3 - Manrique,6.275061,-75.545892
3,Comuna 4 - Aranjuez,6.277723,-75.562599
4,Comuna 5 - Castilla,6.293699,-75.568264
5,Comuna 6 - Doce de Octubre,6.297565,-75.577556
6,Comuna 7 - Robledo,6.279074,-75.588767
7,Comuna 8 - Villa Hermosa,6.253104,-75.550897
8,Comuna 9 - Buenos Aires,6.230779,-75.556594
9,Comuna 10 - La Candelaria,6.250102,-75.567915


In [12]:
address = 'Medellín, Colombia'

geolocator = Nominatim(user_agent="MedellínProject")
location = geolocator.geocode(address, timeout=60, exactly_one=True)
latitude = location.latitude
longitude = location.longitude
print('The decimal coordinates of Bogotá are {}, {}.'.format(latitude, longitude))

The decimal coordinates of Bogotá are 6.2443382, -75.573553.


# make a copy of the dataframe to get it simply back if needed

In [13]:
# make a copy of the dataframe to get it simply back if needed
df3 = df2.copy()
df3

,Comunas,Latitude,Longitude
0,Comuna 1 - Popular,6.293344,-75.544825
1,Comuna 2 - Santa Cruz,6.296956,-75.554852
2,Comuna 3 - Manrique,6.275061,-75.545892
3,Comuna 4 - Aranjuez,6.277723,-75.562599
4,Comuna 5 - Castilla,6.293699,-75.568264
5,Comuna 6 - Doce de Octubre,6.297565,-75.577556
6,Comuna 7 - Robledo,6.279074,-75.588767
7,Comuna 8 - Villa Hermosa,6.253104,-75.550897
8,Comuna 9 - Buenos Aires,6.230779,-75.556594
9,Comuna 10 - La Candelaria,6.250102,-75.567915


In [14]:
"""
    create map of Bogotá using latitude and longitude values
"""
map_medellin = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, local in zip(df3['Latitude'], df3['Longitude'], df3['Comunas']):
    label = '{}'.format(local)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_medellin)  
    
map_medellin